# MVA Mini Project 1

Student: Pierre-Louis Guhur - pierre-louis.guhur@ens-paris-saclay.fr

## Simple Classification

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
#from shutil import copyfile
#copyfile(src="../input/mp1.py", dst = "../working/mp1.py")
from mp1 import *

plt.figure(figsize=(10,10))

plt.subplot(131)
im = generate_a_rectangle(10, True)
plt.imshow(im.reshape(72,72), cmap='gray')

plt.subplot(132)
im = generate_a_disk(10)
plt.imshow(im.reshape(72,72), cmap='gray')

plt.subplot(133)
[im, v] = generate_a_triangle(20, False)
plt.imshow(im.reshape(72,72), cmap='gray')

plt.show()

ModuleNotFoundError: No module named 'mp1'

In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

# [X_train, Y_train] = generate_dataset_classification(300, 20)
# [X_test, Y_test] = generate_test_set_classification()